In [1]:
import yaml
from pprint import pprint
import warnings

import torch

from utils import add_to_catalog
from experiment import EXPERIMENT_CATALOG

warnings.filterwarnings("ignore")

Для удобства разработки была сделана попытка сделать как можно более гибкий шаблон экспериментов. Конечно без костылей не обошлось, однако шаблон сильно упростил расширение библиотеки экспериментов.
Шаблон включает в себя несколько похожих этапов:
* чтение данных
* инициализация моделей
* инициализация оптимизатора и т.д.

Т.к. в данной лабораторной работе все эксперименты проходили с одним и тем же датасетом, получилось добавлять новые эксперименты не путем наследования базового класса, а с помощью интерфейса Train_stage (идейно это общий интерфейс, на практике интерфейс+изолента :D). Т.е. создавались новые Stage'ы и компоновались с помощью ComposeStage, где прогонялись последовательно.

Для логирования использовалась библиотека wandb. Если был запущен где-то wandb.init(), то графики будут логироваться, иначе будет принтится основной лосс на трейне и валидации. Также модели сохраняются в папку с именем run'а из wandb, что может быть достаточно удобным. Так же в папку run'а сохранялся конфиг + мета информация (скорость инференса, время обучения, блеу)  

![scheme](image/experiment.png)

In [3]:
pprint(EXPERIMENT_CATALOG)

{'baseline': <class 'experiment.baseline.Baseline'>,
 'bpe': <class 'experiment.bpe.Baseline'>,
 'pretrain_baseline': <class 'experiment.baseline.Baseline'>,
 'scst': <class 'experiment.scst.SelfCriticalSeqTrain'>}


In [2]:
config_path = 'configs/scst.yaml'
with open(config_path) as fin:
    config = yaml.load(fin)
    
pprint(config)

{'data': {'batch_size': 64,
          'path': '../../datasets/Machine_translation_EN_RU/data.txt',
          'test_size': 0.05,
          'train_size': 0.8,
          'val_size': 0.15,
          'word_min_freq': 5},
 'model': {'name': 'lstm_teacher',
           'params': {'decoder': {'dropout': 0.2,
                                  'emb_dim': 512,
                                  'hid_dim': 512,
                                  'n_layers': 2},
                      'encoder': {'dropout': 0.2,
                                  'emb_dim': 512,
                                  'hid_dim': 512,
                                  'n_layers': 2},
                      'hid_dim': 512,
                      'n_layers': 2,
                      'teacher_forcing_ratio': 0}},
 'model_path': 'model_save/pretrain_baseline_2jxtl023/final-model.pt',
 'pretrain': {'epoch': 5,
              'grad_clip': 1,
              'opt_class': 'Adam',
              'scheduler_class': 'OneCycleLR',
             

Конфиг к каждом стэйджу подтягивается по имени стэйджа. В каждом стэйдже свой оптимизатор и скедулер. Какой оптимизатор или скедулер будет использован можно контролировать через конфиг.

То же самое с моделью. С помощью декоратора utils.add_to_catalog модели добавлялись в словарь, который используется в Experiment классе при ините модели.

In [4]:
TEST_MOD = True
if TEST_MOD:
    config['data']['path'] = 'test_data.txt'
    config.pop('model_path')
experiment_name = 'scst'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
experiment = EXPERIMENT_CATALOG[experiment_name](config, device)

In [5]:
experiment.train()

100%|██████████| 1/1 [00:00<00:00, 33.34it/s]


Epoch: 01
	Train Loss: 2.891 | Train PPL:  18.010
	 Val. Loss: 2.900 |  Val. PPL:  18.170 |  BLEU: 0.000
Epoch: 02
	Train Loss: 2.886 | Train PPL:  17.929
	 Val. Loss: 2.900 |  Val. PPL:  18.170 |  BLEU: 0.000
Epoch: 03
	Train Loss: 2.882 | Train PPL:  17.854
	 Val. Loss: 2.900 |  Val. PPL:  18.170 |  BLEU: 0.000
Epoch: 04
	Train Loss: 2.875 | Train PPL:  17.728
	 Val. Loss: 2.900 |  Val. PPL:  18.169 |  BLEU: 0.000
Epoch: 05
	Train Loss: 2.871 | Train PPL:  17.655
	 Val. Loss: 2.900 |  Val. PPL:  18.169 |  BLEU: 0.000


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 01
	Train Loss: 3.056 | Train PPL:  21.250
	 Val. Loss: 3.032 |  Val. PPL:  20.739 |  BLEU: 0.000
Epoch: 02
	Train Loss: 3.050 | Train PPL:  21.107
	 Val. Loss: 3.033 |  Val. PPL:  20.765 |  BLEU: 0.000
Epoch: 03
	Train Loss: 3.043 | Train PPL:  20.965
	 Val. Loss: 3.036 |  Val. PPL:  20.821 |  BLEU: 0.000


  0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch: 13
	Train Loss: 2.963 | Train PPL:  19.348
	 Val. Loss: 3.053 |  Val. PPL:  21.179 |  BLEU: 0.000
Epoch: 14
	Train Loss: 2.951 | Train PPL:  19.122
	 Val. Loss: 3.056 |  Val. PPL:  21.238 |  BLEU: 0.000
Epoch: 15
	Train Loss: 2.939 | Train PPL:  18.893
	 Val. Loss: 3.059 |  Val. PPL:  21.303 |  BLEU: 0.000


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 01
	Val BLEU: 0.000
Epoch: 02
	Val BLEU: 0.000
Epoch: 03
	Val BLEU: 0.000


100%|██████████| 1/1 [00:00<00:00, 57.55it/s]


Epoch: 04
	Val BLEU: 0.000
Epoch: 05
	Val BLEU: 0.000


In [6]:
experiment.test()

1it [00:00, 61.87it/s]

Bleu: 0.000
